# Set up env

In [11]:
%%sh
# current_path=$(pwd)
# parent_dir=$(dirname "$current_dir")
export PROJECT_HOME="/home/ivan/Uforse/university_crawl"
echo $PROJECT_HOME
if [ -f "${PROJECT_HOME}/build" ]; then
    echo "Found build in PROJECT_HOME. Sourcing now..."
    . "${PROJECT_HOME}/build"
    echo "build executed successfully."
else
    echo "Error: build not found in PROJECT_HOME."
fi

/home/ivan/Uforse/university_crawl
Found build in PROJECT_HOME. Sourcing now...
Obtaining file:///home/ivan/Uforse/university_crawl/university_info_generator
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Attempting uninstall: university-info-generator
    Found existing installation: university-info-generator 0.0.3
    Uninstalling university-info-generator-0.0.3:
      Successfully uninstalled university-info-generator-0.0.3
  Running setup.py develop for university-info-generator
Directory already exists: /home/ivan/Uforse/university_crawl/cache_repo
Environment variables set:
PROJECT_HOME=/home/ivan/Uforse/university_crawl
PYTHONPATH=/home/ivan/Uforse/university_crawl:
API_KEY_PATH=/home/ivan/Uforse/university_crawl
CACHE_REPO_PATH=/home/ivan/Uforse/university_crawl/cache_repo
LANGCHAIN_API_KEY=ls__3780df60c0ef4fd2b2414f032feb31ce
Environment set up for uni_info_generator
build executed successfully.


In [12]:
import os
import sys
from pprint import pprint

#'/home/ivan/Uforse/university_crawl/fetch_logic'
current_path = os.path.abspath('./')

#'/home/ivan/Uforse/university_crawl'
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)
import sys
print(sys.executable)
sys.path.append('/home/ivan/Uforse/university_crawl')

/home/ivan/anaconda3/envs/gpu/bin/python


# Import

In [13]:
import pandas as pd
from pprint import pprint
from typing import Dict
from tqdm import tqdm
from collections import defaultdict
import numpy as np


from university_info_generator import config
from university_info_generator import UniversityInfoGenerator
from university_info_generator import UniversityProgramScraper
from university_info_generator import Program


# Logic

In [21]:
scraper = UniversityProgramScraper()
canada_df = pd.read_csv(f"{scraper.data_repo_path}/canada_dataset/all_universities_canada.csv")
qs_df = pd.read_csv(f"{scraper.data_repo_path}/qs_ranking_2024.csv")

## canada

In [15]:
matched_df = UniversityProgramScraper.join_df(canada_df, qs_df)

In [16]:
scraper = UniversityProgramScraper(need_scrap = True)


In [20]:
canada_info_df = pd.read_csv(f"{scraper.data_repo_path}/canada_dataset/canada_info.csv", index_col="id_")


## scarp on one link

In [ ]:
uwaterloo_program_lst = scraper.get_program_link_lst(link = "https://www.topuniversities.com/universities/university-waterloo")


In [ ]:
program = Program()
for key in Program.valid_keys:
    program.update({key: ""})
program.update({"university_name": "university_name"})


In [ ]:
pprint(program.to_dict_en())

{'academic_degree_level': '',
 'career_direction': '',
 'case_study': '',
 'characteristics': '',
 'coop_opportunity': '',
 'course_requirement': '',
 'degree_type': '',
 'description': '',
 'domestic_student_tuition': '',
 'employment_rate': '',
 'enrollment_language_requirement': '',
 'graduation_rate': '',
 'graduation_year': '',
 'international_student_tuition': '',
 'last_year_condition_to_sophomore': '',
 'location': '',
 'main_subject_area': '',
 'majors_and_employment_direction': '',
 'official_enrollment_score_range': '',
 'others': '',
 'previous_suggested_enrollment_scoreline': '',
 'program_enrollment_number': '',
 'program_id': '',
 'program_name': '',
 'qs_general_program': '',
 'qs_program_link': '',
 'ranking_qs_subject_2024': '',
 'statistics': '',
 'university_id': '',
 'university_name': 'university_name',
 'university_official_website': '',
 'university_type': ''}


In [ ]:
# fetch one university
def fetch_from_program_lst(program_lst: Dict[str, Dict[str, Dict[str, str]]], university_name: str):
    result_dict = {}
    for study_degree in program_lst:
        for qs_general_program in program_lst[study_degree]:
            for program_name in tqdm(program_lst[study_degree][qs_general_program]):
                # print(program_name)
                # program =Program(university_name=university_name)
                program = Program()
                for key in Program.valid_keys:
                    program.update({key: ""})
                link = program_lst[study_degree][qs_general_program][program_name]
                program.update({"university_name": university_name, "qs_program_link": link, "qs_general_program": qs_general_program})
                # program = Program(university_name=university_name)
                program = scraper.extract_program_by_link(link, program=program)
                result_dict[f"{study_degree} {program_name}"] = program.to_dict_en()
    return result_dict

In [ ]:
result = fetch_from_program_lst(uwaterloo_program_lst, "Universty of Waterloo")

  0%|          | 0/20 [00:00<?, ?it/s]

100%|██████████| 29/29 [01:01<00:00,  2.12s/it]


In [ ]:
# for result_program in result:
#     print(result_program)
#     pprint(result[result_program])
#     break

In [ ]:
# df = pd.DataFrame(result.items(), columns=["index"] + list(Program.valid_keys))
# df = pd.DataFrame.from_dict(result)
# df = df[["index"] + list(Program.valid_keys)]
df = pd.DataFrame(result).T

In [ ]:
df.head()

,program_id,university_name,program_name,characteristics,university_official_website,enrollment_language_requirement,employment_rate,career_direction,qs_general_program,last_year_condition_to_sophomore,...,others,program_enrollment_number,ranking_qs_subject_2024,degree_type,official_enrollment_score_range,domestic_student_tuition,coop_opportunity,case_study,graduation_rate,graduation_year
bachelor Architectural Engineering (BAS),,Universty of Waterloo,Architectural Engineering (BAS),,,TOEFL: 90+\nIELTS: 6.5+,,,,,...,,,101-150,Other,,"CAD $64,480",,,,4
bachelor Architecture,,Universty of Waterloo,Architecture,,,TOEFL: 90+\nIELTS: 6.5+,,,,,...,,,101-150,Other,,"CAD $42,080",,,,4
bachelor Classical Studies,,Universty of Waterloo,Classical Studies,,,TOEFL: 90+\nIELTS: 6.5+,,,,,...,,,,BA,,"CAD $28,800",,,,4
bachelor English,,Universty of Waterloo,English,,,TOEFL: 90+\nIELTS: 6.5+,,,,,...,,,251-300,BA,,"CAD $28,800",,,,4
bachelor Fine Arts,,Universty of Waterloo,Fine Arts,,,TOEFL: 90+\nIELTS: 6.5+,,,,,...,,,151-200,BA,,"CAD $28,800",,,,4


In [ ]:
df.to_csv("/home/ivan/Uforse/university_crawl/university_info_generator/fetch_logic/fetch_data/programs/canada/uwaterloo_programs.csv", encoding="utf-8")

## usa


In [ ]:
us_df = pd.read_csv(f"{scraper.data_repo_path}/usa_dataset/all_universities_usa.csv")
matched_usa_df = UniversityProgramScraper.join_df(us_df, qs_df, save_path=f"{config.PROJECT_HOME}/university_info_generator/data/usa_dataset/qs_usa_uni_matched.csv")